<a href="https://colab.research.google.com/github/Limsungrae/2024/blob/main/%EA%B0%95%ED%99%94_%ED%95%99%EC%8A%B5_1805129_%EC%9E%84%EC%84%B1%EB%9E%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 3주 환경과 에이전트 준비/ 교제 p.26-32

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import time
import copy

In [3]:
def show_v_table_small(v_table, env):
    for i in range(env.reward.shape[0]):
        print("+----------"*env.reward.shape[1])
        print("|", end="")
        for j in range(env.reward.shape[1]):
            print("{0:8.2f}  |".format(v_table[i,j]),end="")
        print()
    print("+----------"*env.reward.shape[1])

# V table 그리기
def show_v_table(v_table, env):
    for i in range(env.reward.shape[0]):
        print("+-----------------"*env.reward.shape[1],end="")
        print("+")
        for k in range(3):
            print("|",end="")
            for j in range(env.reward.shape[1]):
                if k==0:
                    print("                 |",end="")
                if k==1:
                        print("   {0:8.2f}      |".format(v_table[i,j]),end="")
                if k==2:
                    print("                 |",end="")
            print()
    print("+-----------------"*env.reward.shape[1],end="")
    print("+")

# Q table 그리기
def show_q_table(q_table,env):
    for i in range(env.reward.shape[0]):
        print("+-----------------"*env.reward.shape[1],end="")
        print("+")
        for k in range(3):
            print("|",end="")
            for j in range(env.reward.shape[1]):
                if k==0:
                    print("{0:10.2f}       |".format(q_table[i,j,0]),end="")
                if k==1:
                    print("{0:6.2f}    {1:6.2f} |".format(q_table[i,j,3],q_table[i,j,1]),end="")
                if k==2:
                    print("{0:10.2f}       |".format(q_table[i,j,2]),end="")
            print()
    print("+-----------------"*env.reward.shape[1],end="")
    print("+")


# 정책 policy 화살표로 그리기
def show_q_table_arrow(q_table,env):
    for i in range(env.reward.shape[0]):
        print("+-----------------"*env.reward.shape[1],end="")
        print("+")
        for k in range(3):
            print("|",end="")
            for j in range(env.reward.shape[1]):
                if k==0:
                    if np.max(q[i,j,:]) == q[i,j,0]:
                        print("        ↑       |",end="")
                    else:
                        print("                 |",end="")
                if k==1:
                    if np.max(q[i,j,:]) == q[i,j,1] and np.max(q[i,j,:]) == q[i,j,3]:
                        print("      ←  →     |",end="")
                    elif np.max(q[i,j,:]) == q[i,j,1]:
                        print("          →     |",end="")
                    elif np.max(q[i,j,:]) == q[i,j,3]:
                        print("      ←         |",end="")
                    else:
                        print("                 |",end="")
                if k==2:
                    if np.max(q[i,j,:]) == q[i,j,2]:
                        print("        ↓       |",end="")
                    else:
                        print("                 |",end="")
            print()
    print("+-----------------"*env.reward.shape[1],end="")
    print("+")

# 정책 policy 화살표로 그리기
def show_policy_small(policy,env):
    for i in range(env.reward.shape[0]):
        print("+----------"*env.reward.shape[1],end="")
        print("+")
        print("|", end="")
        for j in range(env.reward.shape[1]):
            if env.reward_list1[i][j] == "road":
                if policy[i,j] == 0:
                    print("   ↑     |",end="")
                elif policy[i,j] == 1:
                    print("   →     |",end="")
                elif policy[i,j] == 2:
                    print("   ↓     |",end="")
                elif policy[i,j] == 3:
                    print("   ←     |",end="")
            else:
                print("          |",end="")
        print()
    print("+----------"*env.reward.shape[1],end="")
    print("+")

# 정책 policy 화살표로 그리기
def show_policy(policy,env):
    for i in range(env.reward.shape[0]):
        print("+-----------------"*env.reward.shape[1],end="")
        print("+")
        for k in range(3):
            print("|",end="")
            for j in range(env.reward.shape[1]):
                if k==0:
                    print("                 |",end="")
                if k==1:
                    if policy[i,j] == 0:
                        print("      ↑         |",end="")
                    elif policy[i,j] == 1:
                        print("      →         |",end="")
                    elif policy[i,j] == 2:
                        print("      ↓         |",end="")
                    elif policy[i,j] == 3:
                        print("      ←         |",end="")
                if k==2:
                    print("                 |",end="")
            print()
    print("+-----------------"*env.reward.shape[1],end="")
    print("+")

### 31p

In [4]:
class Agent():

    # 1. 행동에 따른 에이전트의 좌표 이동(위, 오른쪽, 아래, 왼쪽)
    action = np.array([[-1,0],[0,1],[1,0],[0,-1]])

    # 2. 각 행동별 선택확률
    select_action_pr = np.array([0.25,0.25,0.25,0.25])

    # 3. 에이전트의 초기 위치 저장
    def __init__(self):
        self.pos = (0,0)

    # 4. 에이전트의 위치 저장
    def set_pos(self,position):
        self.pos = position
        return self.pos

    # 5. 에이전트의 위치 불러오기
    def get_pos(self):
        return self.pos

### Environment 구현 28,30p

In [5]:
class Environment():

    # 1. 미로밖(절벽), 길, 목적지와 보상 설정
    cliff = -3
    road = -1
    goal = 1

    # 2. 목적지 좌표 설정
    goal_position = [2,2]

    # 3. 보상 리스트 숫자
    reward_list = [[road,road,road],
                   [road,road,road],
                   [road,road,goal]]

    # 4. 보상 리스트 문자
    reward_list1 = [["road","road","road"],
                    ["road","road","road"],
                    ["road","road","goal"]]

    # 5. 보상 리스트를 array로 설정
    def __init__(self):
        self.reward = np.asarray(self.reward_list)

    # 6. 선택된 에이전트의 행동 결과 반환 (미로밖일 경우 이전 좌표로 다시 복귀)
    def move(self, agent, action):

        done = False

        # 6.1 행동에 따른 좌표 구하기
        new_pos = agent.pos + agent.action[action]

        # 6.2 현재좌표가 목적지 인지확인
        if self.reward_list1[agent.pos[0]][agent.pos[1]] == "goal":
            reward = self.goal
            observation = agent.set_pos(agent.pos)
            done = True
        # 6.3 이동 후 좌표가 미로 밖인 확인
        elif new_pos[0] < 0 or new_pos[0] >= self.reward.shape[0] or new_pos[1] < 0 or new_pos[1] >= self.reward.shape[1]:
            reward = self.cliff
            observation = agent.set_pos(agent.pos)
            done = True
        # 6.4 이동 후 좌표가 길이라면
        else:
            observation = agent.set_pos(new_pos)
            reward = self.reward[observation[0],observation[1]]

        return observation, reward, done

### 상태가치 ㅍ함수 교제 40~43

In [6]:
# 상태 가치 계산
def state_value_function(env,agent,G,max_step,now_step):

    # 1. 감가율 설정
    gamma = 0.9

# 2. 현재 위치가 도착지점인지 확인
    if env.reward_list1[agent.pos[0]][agent.pos[1]] == "goal":
        return env.goal

# 3. 마지막 상태는 보상만 계산
    if (max_step == now_step):
        pos1 = agent.get_pos()

        # 3.1 가능한 모든 행동의 보상을 계산
        for i in range(len(agent.action)):
            agent.set_pos(pos1)
            observation, reward, done = env.move(agent,i)
            G += agent.select_action_pr[i] * reward

        return G

    # 4. 현재 상태의 보상을 계산한 후 다음 step으로 이동
    else:

        # 4.1현재 위치 저장
        pos1 = agent.get_pos()

        # 4.2 현재 위치에서 가능한 모든 행동을 조사한 후 이동
        for i in range(len(agent.action)):
            observation, reward, done = env.move(agent,i)
            # 4.2.1 현재 상태에서 보상을 계산
            G += agent.select_action_pr[i] * reward

            # 4.2.2 이동 후 위치 확인 : 미로밖, 벽, 구멍인 경우 이동전 좌표로 다시 이동
            if done == True:
                if observation[0] < 0 or observation[0] >= env.reward.shape[0] or observation[1] < 0 or observation[1] >= env.reward.shape[1]:
                    agent.set_pos(pos1)

            # 4.2.3 다음 step을 계산
            next_v = state_value_function(env, agent, 0, max_step, now_step+1)
            G += agent.select_action_pr[i] * gamma * next_v

            # 4.2.4 현재 위치를 복구
            agent.set_pos(pos1)

        return G


### 42~43p

In [ ]:
# 1. 환경 초기화
env = Environment()

# 2. 에이전트 초기화
agent = Agent()

# 3. 최대 max_step_number 제한
max_step_number = 13

# 4. 계산 시간 저장을 위한 list
time_len = []

# 5. 재귀함수 state_value_function을를 이용해 각 상태 가치를 계산
for max_step in range(max_step_number):

    # 5.1 미로 각 상태의 가치를 테이블 형식으로 저장
    v_table = np.zeros((env.reward.shape[0],env.reward.shape[1]))
    start_time = time.time()

    # 5.2 미로의 각 상태에 대해 state_value_function() 을 이용해 가치를 계산한 후 테이블 형식으로 저장
    for i in range(env.reward.shape[0]):
        for j in range(env.reward.shape[1]):
            agent.set_pos([i,j])
            v_table[i,j] = state_value_function(env,agent, 0, max_step, 0)

    # 5.3 max_down에 따른 계산시간 저장
    time_len.append(time.time()-start_time)
    print("max_step_number = {} total_time = {}(s)".format(max_step, np.round(time.time()-start_time,2)))

    show_v_table(np.round(v_table,2),env)

# 6. step 별 계산 시간 그래프 그리기
plt.plot(time_len, 'o-k')
plt.xlabel('max_down')
plt.ylabel('time(s)')
plt.legend()
plt.show()

max_step_number = 0 total_time = 0.0(s)
+-----------------+-----------------+-----------------+
|                 |                 |                 |
|      -2.00      |      -1.50      |      -2.00      |
|                 |                 |                 |
+-----------------+-----------------+-----------------+
|                 |                 |                 |
|      -1.50      |      -1.00      |      -1.00      |
|                 |                 |                 |
+-----------------+-----------------+-----------------+
|                 |                 |                 |
|      -2.00      |      -1.00      |       1.00      |
|                 |                 |                 |
+-----------------+-----------------+-----------------+
max_step_number = 1 total_time = 0.0(s)
+-----------------+-----------------+-----------------+
|                 |                 |                 |
|      -3.58      |      -2.96      |      -3.46      |
|                 |     

### 5주차 행동가치함수 교제 p49~51

In [ ]:
# 행동 가치 함수
def action_value_function(env, agent, act, G, max_step, now_step):

    # 1. 감가율 설정
    gamma = 0.9

    # 2. 현재 위치가 목적지인지 확인
    if env.reward_list1[agent.pos[0]][agent.pos[1]] == "goal":
        return env.goal

    # 3. 마지막 상태는 보상만 계산
    if (max_step == now_step):
        observation, reward, done = env.move(agent, act)
        G += agent.select_action_pr[act]*reward
        return G

    # 4. 현재 상태의 보상을 계산한 후 다음 행동과 함께 다음 step으로 이동
    else:
        # 4.1현재 위치 저장
        pos1 = agent.get_pos()
        observation, reward, done = env.move(agent, act)
        G += agent.select_action_pr[act] * reward

        # 4.2 이동 후 위치 확인 : 미로밖, 벽, 구멍인 경우 이동전 좌표로 다시 이동
        if done == True:
            if observation[0] < 0 or observation[0] >= env.reward.shape[0] or observation[1] < 0 or observation[1] >= env.reward.shape[1]:
                agent.set_pos(pos1)

        # 4.3 현재 위치를 다시 저장
        pos1 = agent.get_pos()

        # 4.4 현재 위치에서 가능한 모든 행동을 선택한 후 이동
        for i in range(len(agent.action)):
            agent.set_pos(pos1)
            next_v = action_value_function(env, agent, i, 0, max_step, now_step+1)
            G += agent.select_action_pr[i] * gamma * next_v
        return G


### 51p .미로의 각 상태의 행동가치함수를 구하는 함수

In [ ]:
# 재귀적으로 행동의 가치를 계산

# 1. 환경 초기화
env = Environment()

# 2. 에이전트 초기화
agent = Agent()
np.random.seed(0)

# 3. 현재부터 max_step 까지 계산
max_step_number = 8

# 4. 모든 상태에 대해
for max_step in range(max_step_number):
    # 4.1 미로 상의 모든 상태에서 가능한 행동의 가치를 저장할 테이블을 정의
    print("max_step = {}".format(max_step))
    q_table = np.zeros((env.reward.shape[0], env.reward.shape[1],len(agent.action)))
    for i in range(env.reward.shape[0]):
        for j in range(env.reward.shape[1]):
            # 4.2 모든 행동에 대해
            for action in range(len(agent.action)):
                # 4.2.1 에이전트의 위치를 초기화
                agent.set_pos([i,j])
                # 4.2.2 현재 위치에서 행동 가치를 계산
                q_table[i ,j,action] = action_value_function(env, agent, action, 0, max_step, 0)

    q = np.round(q_table,2)
    print("Q - table")
    show_q_table(q, env)
    print("High actions Arrow")
    show_q_table_arrow(q,env)
    print()

### 6주차 동적계획법 : 최적정책선택(1)/교제 p60-61

In [ ]:
# 반복 정책 평가
np.random.seed(0)
env = Environment()
agent = Agent()
gamma = 0.9

# 1. 모든 𝑠∈𝑆^에 대해서 배열 𝑉(𝑠)=0으로 초기화
v_table = np.zeros((env.reward.shape[0],env.reward.shape[1]))

print("start Iterative Policy Evaluation")

k = 1
print()
print("V0(S)   k = 0")

# 초기화된 V 테이블 출력
show_v_table(np.round(v_table,2),env)

# 시작 시간 변수에 저장
start_time = time.time()

# 반복
while(True):
    # 2. Δ←0
    delta = 0
    # 3. v←(𝑠)
    # 계산전 가치를 저장
    temp_v = copy.deepcopy(v_table)
    # 4. 모든 𝑠∈𝑆에 대해 :
    for i in range(env.reward.shape[0]):
        for j in range(env.reward.shape[1]):
            G = 0
            # 5. 가능한 모든 행동으로 다음상태만 이용해 𝑉(𝑠) 계산
            for action in range(len(agent.action)):
                agent.set_pos([i,j])
                observation, reward, done = env.move(agent, action)

#                 print("s({0}): {1:5s} : {2:0.2f} = {3:0.2f} *({4:0.2f} +  {5:0.2f} *  {6:0.2f})".format(i*env.reward.shape[0]+j,dic[action],agent.select_action_pr[action] * (reward + gamma*V[observation[0],observation[1]]), agent.select_action_pr[action],reward,gamma,V[observation[0],observation[1]]))

                G += agent.select_action_pr[action] * (reward + gamma*v_table[observation[0],observation[1]])

#             print("V{2}({0}) :sum = {1:.2f}".format(i*env.reward.shape[0]+j,total,k))
#             print()
            v_table[i,j] = G
    # 6. ∆←max⁡(∆,|v−𝑉(𝑠)|)
    # 계산전과 계산후의 가치 차이 계산
    delta = np.max([delta, np.max(np.abs(temp_v-v_table))])

    end_time = time.time()
    print("V{0}(S) : k = {1:3d}    delta = {2:0.6f} total_time = {3}".format(k,k, delta,np.round(end_time-start_time),2))
    show_v_table(np.round(v_table,2),env)
    k +=1

    # 7. ∆ <𝜃가 작은 양수 일 때까지 반복

    if delta < 0.000001:
        break

end_time = time.time()
print("total_time = {}".format(np.round(end_time-start_time),2))



start Iterative Policy Evaluation

V0(S)   k = 0
+-----------------+-----------------+-----------------+
|                 |                 |                 |
|       0.00      |       0.00      |       0.00      |
|                 |                 |                 |
+-----------------+-----------------+-----------------+
|                 |                 |                 |
|       0.00      |       0.00      |       0.00      |
|                 |                 |                 |
+-----------------+-----------------+-----------------+
|                 |                 |                 |
|       0.00      |       0.00      |       0.00      |
|                 |                 |                 |
+-----------------+-----------------+-----------------+
V1(S) : k =   1    delta = 2.438750 total_time = 0.0
+-----------------+-----------------+-----------------+
|                 |                 |                 |
|      -2.00      |      -1.95      |      -2.44      |
| 

|                 |                 |                 |
+-----------------+-----------------+-----------------+
V21(S) : k =  21    delta = 0.121577 total_time = 1.0
+-----------------+-----------------+-----------------+
|                 |                 |                 |
|     -12.14      |     -10.35      |      -9.41      |
|                 |                 |                 |
+-----------------+-----------------+-----------------+
|                 |                 |                 |
|     -10.35      |      -7.31      |      -3.61      |
|                 |                 |                 |
+-----------------+-----------------+-----------------+
|                 |                 |                 |
|      -9.41      |      -3.61      |       8.91      |
|                 |                 |                 |
+-----------------+-----------------+-----------------+
V22(S) : k =  22    delta = 0.109419 total_time = 1.0
+-----------------+-----------------+---------------

|                 |                 |                 |
|      -8.95      |      -3.02      |       9.87      |
|                 |                 |                 |
+-----------------+-----------------+-----------------+
V42(S) : k =  42    delta = 0.013303 total_time = 1.0
+-----------------+-----------------+-----------------+
|                 |                 |                 |
|     -11.79      |      -9.94      |      -8.95      |
|                 |                 |                 |
+-----------------+-----------------+-----------------+
|                 |                 |                 |
|      -9.94      |      -6.83      |      -3.01      |
|                 |                 |                 |
+-----------------+-----------------+-----------------+
|                 |                 |                 |
|      -8.95      |      -3.01      |       9.88      |
|                 |                 |                 |
+-----------------+-----------------+-------------

|                 |                 |                 |
+-----------------+-----------------+-----------------+
|                 |                 |                 |
|      -8.88      |      -2.94      |       9.99      |
|                 |                 |                 |
+-----------------+-----------------+-----------------+
V63(S) : k =  63    delta = 0.001456 total_time = 2.0
+-----------------+-----------------+-----------------+
|                 |                 |                 |
|     -11.72      |      -9.88      |      -8.88      |
|                 |                 |                 |
+-----------------+-----------------+-----------------+
|                 |                 |                 |
|      -9.88      |      -6.77      |      -2.94      |
|                 |                 |                 |
+-----------------+-----------------+-----------------+
|                 |                 |                 |
|      -8.88      |      -2.94      |       9.99  

|                 |                 |                 |
|      -9.88      |      -6.76      |      -2.93      |
|                 |                 |                 |
+-----------------+-----------------+-----------------+
|                 |                 |                 |
|      -8.87      |      -2.93      |      10.00      |
|                 |                 |                 |
+-----------------+-----------------+-----------------+
V84(S) : k =  84    delta = 0.000159 total_time = 3.0
+-----------------+-----------------+-----------------+
|                 |                 |                 |
|     -11.72      |      -9.88      |      -8.87      |
|                 |                 |                 |
+-----------------+-----------------+-----------------+
|                 |                 |                 |
|      -9.88      |      -6.76      |      -2.93      |
|                 |                 |                 |
+-----------------+-----------------+-------------

|                 |                 |                 |
+-----------------+-----------------+-----------------+
|                 |                 |                 |
|      -9.88      |      -6.76      |      -2.93      |
|                 |                 |                 |
+-----------------+-----------------+-----------------+
|                 |                 |                 |
|      -8.87      |      -2.93      |      10.00      |
|                 |                 |                 |
+-----------------+-----------------+-----------------+
V105(S) : k = 105    delta = 0.000017 total_time = 4.0
+-----------------+-----------------+-----------------+
|                 |                 |                 |
|     -11.72      |      -9.88      |      -8.87      |
|                 |                 |                 |
+-----------------+-----------------+-----------------+
|                 |                 |                 |
|      -9.88      |      -6.76      |      -2.93 

|                 |                 |                 |
|     -11.72      |      -9.88      |      -8.87      |
|                 |                 |                 |
+-----------------+-----------------+-----------------+
|                 |                 |                 |
|      -9.88      |      -6.76      |      -2.93      |
|                 |                 |                 |
+-----------------+-----------------+-----------------+
|                 |                 |                 |
|      -8.87      |      -2.93      |      10.00      |
|                 |                 |                 |
+-----------------+-----------------+-----------------+
V126(S) : k = 126    delta = 0.000002 total_time = 4.0
+-----------------+-----------------+-----------------+
|                 |                 |                 |
|     -11.72      |      -9.88      |      -8.87      |
|                 |                 |                 |
+-----------------+-----------------+------------

### 7주차 동적꼐획법 : 최적정책선택(2) / 교재 p.66~73

In [1]:
def policy_evalution(env, agent, v_table, policy):
    gamma = 0.9
    while(True):
        # Δ←0
        delta = 0
        #  v←𝑉(𝑠)
        temp_v = copy.deepcopy(v_table)
        # 모든 𝑠∈𝑆에 대해 :
        for i in range(env.reward.shape[0]):
            for j in range(env.reward.shape[1]):
                # 에이전트를 지정된 좌표에 위치시킨후 가치함수를 계산
                agent.set_pos([i,j])
                # 현재 정책의 행동을 선택
                action = policy[i,j]
                observation, reward, done = env.move(agent, action)
                v_table[i,j] = reward + gamma * v_table[observation[0],observation[1]]
        # ∆←max⁡(∆,|v−𝑉(𝑠)|)
        # 계산전과 계산후의 가치의 차이를 계산
        delta = np.max([delta, np.max(np.abs(temp_v-v_table))])

        # 7. ∆ <𝜃가 작은 양수 일 때까지 반복
        if delta < 0.000001:
            break
    return v_table, delta


def policy_improvement(env, agent, v_table, policy):

    # 67페이지 아래 누락 되어있습니다
    gamma = 0.9

    # policyStable ← true
    policyStable = True

    # 모든 s∈S에 대해：
    for i in range(env.reward.shape[0]):
        for j in range(env.reward.shape[1]):
            # 𝑜𝑙𝑑−𝑎𝑐𝑡𝑖𝑜𝑛←π(s)
            old_action = policy[i,j]
            # 가능한 행동중 최댓값을 가지는 행동을 선택
            temp_action = 0
            temp_value =  -1e+10
            for action in range(len(agent.action)):
                agent.set_pos([i,j])
                observation, reward, done = env.move(agent,action)
                if temp_value < reward + gamma * v_table[observation[0],observation[1]]:
                    temp_action = action
                    temp_value = reward + gamma * v_table[observation[0],observation[1]]
            # 만약 𝑜𝑙𝑑−𝑎𝑐𝑡𝑖𝑜𝑛"≠π(s)"라면， "policyStable ← False"
            # old-action과 새로운 action이 다른지 체크
            if old_action != temp_action :
                policyStable = False
            policy[i,j] = temp_action
    return policy, policyStable

# 정책 반복
# 환경과 에이전트에 대한 초기 설정
np.random.seed(0)
env = Environment()
agent = Agent()

# 1. 초기화
# 모든 𝑠∈𝑆에 대해 𝑉(𝑠)∈𝑅과 π(𝑠)∈𝐴(𝑠)를 임의로 설정
v_table =  np.random.rand(env.reward.shape[0], env.reward.shape[1])
policy = np.random.randint(0, 4,(env.reward.shape[0], env.reward.shape[1]))

print("Initial random V(S)")
show_v_table(np.round(v_table,2),env)
print()
print("Initial random Policy π0(S)")
show_policy(policy,env)
print("start policy iteration")

# 시작 시간을 변수에 저장
start_time = time.time()

max_iter_number = 20000
for iter_number in range(max_iter_number):

    # 2.정책평가
    v_table, delta = policy_evalution(env, agent, v_table, policy)

    # 정책 평가 후 결과 표시
    print("")
    print("Vπ{0:}(S) delta = {1:.10f}".format(iter_number,delta))
    show_v_table(np.round(v_table,2),env)
    print()


    # 3.정책개선
    policy, policyStable = policy_improvement(env, agent, v_table, policy)

    # policy 변화 저장
    print("policy π{}(S)".format(iter_number+1))
    show_policy(policy,env)
    # 하나라도 old-action과 새로운 action이 다르다면 '2. 정책평가'를 반복
    if(policyStable == True):
        break


print("total_time = {}".format(time.time()-start_time))


NameError: name 'np' is not defined

### 9주차 몬테카롤로 방법 교제 p.81~93

In [ ]:
def finding_optimal_value_function(env, agent, v_table):
    k = 1
    gamma = 0.9
    while(True):
        # Δ←0
        delta=0
        #  v←𝑉(𝑠)
        temp_v = copy.deepcopy(v_table)

        # 모든 𝑠∈𝑆에 대해 :
        for i in range(env.reward.shape[0]):
            for j in range(env.reward.shape[1]):
                temp = -1e+10
#                 print("s({0}):".format(i*env.reward.shape[0]+j))
                # 𝑉(𝑠)← max(a)⁡∑𝑃(𝑠'|𝑠,𝑎)[𝑟(𝑠,𝑎,𝑠') +𝛾𝑉(𝑠')]
                # 가능한 행동을 선택
                for action in range(len(agent.action)):
                    agent.set_pos([i,j])
                    observation, reward, done = env.move(agent, action)
#                     print("{0:.2f} = {1:.2f} + {2:.2f} * {3:.2f}" .format(reward + gamma* v_table[observation[0],observation[1]],reward, gamma,v_table[observation[0],observation[1]]))
                    #이동한 상태의 가치가 temp보다 크면
                    if temp < reward + gamma*v_table[observation[0],observation[1]]:
                        # temp 에 새로운 가치를 저장
                        temp = reward + gamma*v_table[observation[0],observation[1]]
#                 print("V({0}) :max = {1:.2f}".format(i*env.reward.shape[0]+j,temp))
#                 print()
                # 이동 가능한 상태 중 가장 큰 가치를 저장
                v_table[i,j] = temp

        #  ∆←max⁡(∆,|v−𝑉(𝑠)|)
        # 이전 가치와 비교해서 큰 값을 delta에 저장
        # 계산전과 계산후의 가치의 차이 계산
        delta = np.max([delta, np.max(np.abs(temp_v-v_table))])
        # 7. ∆ <𝜃가 작은 양수 일 때까지 반복
        if delta < 0.0000001:
            break

#         if k < 4 or k > 150:
#             print("V{0}(S) : k = {1:3d}    delta = {2:0.6f}".format(k,k, delta))
#             show_v_table(np.round(v_table,2),env)
        print("V{0}(S) : k = {1:3d}    delta = {2:0.6f}".format(k,k, delta))
        show_v_table(np.round(v_table,2),env)
        k +=1

    return v_table

def policy_extraction(env, agent, v_table, optimal_policy):

    gamma = 0.9

    #정책 𝜋를 다음과 같이 추출
    # 𝜋(𝑠)← argmax(a)⁡∑𝑃(𝑠'|𝑠,𝑎)[𝑟(𝑠,𝑎,𝑠') +𝛾𝑉(𝑠')]
    # 모든 𝑠∈𝑆에 대해 :
    for i in range(env.reward.shape[0]):
        for j in range(env.reward.shape[1]):
            temp =  -1e+10
            # 가능한 행동중 가치가 가장높은 값을 policy[i,j]에 저장
            for action in range(len(agent.action)):
                agent.set_pos([i,j])
                observation, reward, done = env.move(agent,action)
                if temp < reward + gamma * v_table[observation[0],observation[1]]:
                    optimal_policy[i,j] = action
                    temp = reward + gamma * v_table[observation[0],observation[1]]

    return optimal_policy


# 가치 반복

# 환경, 에이전트를 초기화
np.random.seed(0)
env = Environment()
agent = Agent()

# 초기화
# 모든 𝑠∈𝑆^+에 대해 𝑉(𝑠)∈𝑅을 임의로 설정
v_table =  np.random.rand(env.reward.shape[0], env.reward.shape[1])

print("Initial random V0(S)")
show_v_table(np.round(v_table,2),env)
print()

optimal_policy = np.zeros((env.reward.shape[0], env.reward.shape[1]))

print("start Value iteration")
print()

# 시작 시간 변수에 저장
start_time = time.time()

v_table = finding_optimal_value_function(env, agent, v_table)

optimal_policy = policy_extraction(env, agent, v_table, optimal_policy)


print("total_time = {}".format(np.round(time.time()-start_time),2))
print()
print("Optimal policy")
show_policy(optimal_policy, env)


Initial random V0(S)
+-----------------+-----------------+-----------------+
|                 |                 |                 |
|       0.55      |       0.72      |       0.60      |
|                 |                 |                 |
+-----------------+-----------------+-----------------+
|                 |                 |                 |
|       0.54      |       0.42      |       0.65      |
|                 |                 |                 |
+-----------------+-----------------+-----------------+
|                 |                 |                 |
|       0.44      |       0.89      |       0.96      |
|                 |                 |                 |
+-----------------+-----------------+-----------------+

start Value iteration

V1(S) : k =   1    delta = 1.221402
+-----------------+-----------------+-----------------+
|                 |                 |                 |
|      -0.36      |      -0.46      |      -0.42      |
|                 |    

|                 |                 |                 |
|       4.85      |       6.85      |       6.85      |
|                 |                 |                 |
+-----------------+-----------------+-----------------+
V11(S) : k =  11    delta = 0.315078
+-----------------+-----------------+-----------------+
|                 |                 |                 |
|       1.74      |       3.36      |       5.16      |
|                 |                 |                 |
+-----------------+-----------------+-----------------+
|                 |                 |                 |
|       3.36      |       5.16      |       7.16      |
|                 |                 |                 |
+-----------------+-----------------+-----------------+
|                 |                 |                 |
|       5.16      |       7.16      |       7.16      |
|                 |                 |                 |
+-----------------+-----------------+-----------------+
V12(S) : k 

|                 |                 |                 |
|       5.21      |       7.01      |       9.01      |
|                 |                 |                 |
+-----------------+-----------------+-----------------+
|                 |                 |                 |
|       7.01      |       9.01      |       9.01      |
|                 |                 |                 |
+-----------------+-----------------+-----------------+
V22(S) : k =  22    delta = 0.098875
+-----------------+-----------------+-----------------+
|                 |                 |                 |
|       3.69      |       5.31      |       7.11      |
|                 |                 |                 |
+-----------------+-----------------+-----------------+
|                 |                 |                 |
|       5.31      |       7.11      |       9.11      |
|                 |                 |                 |
+-----------------+-----------------+-----------------+
|          

|                 |                 |                 |
|       4.27      |       5.89      |       7.69      |
|                 |                 |                 |
+-----------------+-----------------+-----------------+
|                 |                 |                 |
|       5.89      |       7.69      |       9.69      |
|                 |                 |                 |
+-----------------+-----------------+-----------------+
|                 |                 |                 |
|       7.69      |       9.69      |       9.69      |
|                 |                 |                 |
+-----------------+-----------------+-----------------+
V33(S) : k =  33    delta = 0.031028
+-----------------+-----------------+-----------------+
|                 |                 |                 |
|       4.30      |       5.92      |       7.72      |
|                 |                 |                 |
+-----------------+-----------------+-----------------+
|          

|       7.89      |       9.89      |       9.89      |
|                 |                 |                 |
+-----------------+-----------------+-----------------+
V43(S) : k =  43    delta = 0.010819
+-----------------+-----------------+-----------------+
|                 |                 |                 |
|       4.48      |       6.10      |       7.90      |
|                 |                 |                 |
+-----------------+-----------------+-----------------+
|                 |                 |                 |
|       6.10      |       7.90      |       9.90      |
|                 |                 |                 |
+-----------------+-----------------+-----------------+
|                 |                 |                 |
|       7.90      |       9.90      |       9.90      |
|                 |                 |                 |
+-----------------+-----------------+-----------------+
V44(S) : k =  44    delta = 0.009737
+-----------------+-----------

...

+-----------------+-----------------+-----------------+
V107(S) : k = 107    delta = 0.000013
+-----------------+-----------------+-----------------+
|                 |                 |                 |
|       4.58      |       6.20      |       8.00      |
|                 |                 |                 |
+-----------------+-----------------+-----------------+
|                 |                 |                 |
|       6.20      |       8.00      |      10.00      |
|                 |                 |                 |
+-----------------+-----------------+-----------------+
|                 |                 |                 |
|       8.00      |      10.00      |      10.00      |
|                 |                 |                 |
+-----------------+-----------------+-----------------+
V108(S) : k = 108    delta = 0.000011
+-----------------+-----------------+-----------------+
|                 |                 |                 |
|       4.58      |       6.

|                 |                 |                 |
+-----------------+-----------------+-----------------+
|                 |                 |                 |
|       8.00      |      10.00      |      10.00      |
|                 |                 |                 |
+-----------------+-----------------+-----------------+
V118(S) : k = 118    delta = 0.000004
+-----------------+-----------------+-----------------+
|                 |                 |                 |
|       4.58      |       6.20      |       8.00      |
|                 |                 |                 |
+-----------------+-----------------+-----------------+
|                 |                 |                 |
|       6.20      |       8.00      |      10.00      |
|                 |                 |                 |
+-----------------+-----------------+-----------------+
|                 |                 |                 |
|       8.00      |      10.00      |      10.00      |
|         

|       4.58      |       6.20      |       8.00      |
|                 |                 |                 |
+-----------------+-----------------+-----------------+
|                 |                 |                 |
|       6.20      |       8.00      |      10.00      |
|                 |                 |                 |
+-----------------+-----------------+-----------------+
|                 |                 |                 |
|       8.00      |      10.00      |      10.00      |
|                 |                 |                 |
+-----------------+-----------------+-----------------+
V129(S) : k = 129    delta = 0.000001
+-----------------+-----------------+-----------------+
|                 |                 |                 |
|       4.58      |       6.20      |       8.00      |
|                 |                 |                 |
+-----------------+-----------------+-----------------+
|                 |                 |                 |
|       6.

|                 |                 |                 |
+-----------------+-----------------+-----------------+
V139(S) : k = 139    delta = 0.000000
+-----------------+-----------------+-----------------+
|                 |                 |                 |
|       4.58      |       6.20      |       8.00      |
|                 |                 |                 |
+-----------------+-----------------+-----------------+
|                 |                 |                 |
|       6.20      |       8.00      |      10.00      |
|                 |                 |                 |
+-----------------+-----------------+-----------------+
|                 |                 |                 |
|       8.00      |      10.00      |      10.00      |
|                 |                 |                 |
+-----------------+-----------------+-----------------+
V140(S) : k = 140    delta = 0.000000
+-----------------+-----------------+-----------------+
|                 |         

|       6.20      |       8.00      |      10.00      |
|                 |                 |                 |
+-----------------+-----------------+-----------------+
|                 |                 |                 |
|       8.00      |      10.00      |      10.00      |
|                 |                 |                 |
+-----------------+-----------------+-----------------+
V150(S) : k = 150    delta = 0.000000
+-----------------+-----------------+-----------------+
|                 |                 |                 |
|       4.58      |       6.20      |       8.00      |
|                 |                 |                 |
+-----------------+-----------------+-----------------+
|                 |                 |                 |
|       6.20      |       8.00      |      10.00      |
|                 |                 |                 |
+-----------------+-----------------+-----------------+
|                 |                 |                 |
|       8.

###81~82

In [ ]:
def generate_episode(env, agent, first_visit):
    gamma = 0.09
    # 에피소드를 저장할 리스트
    episode = []
    # 이전에 방문여부 체크
    visit = np.zeros((env.reward.shape[0], env.reward.shape[1]))

    # 에이전트가 모든 상태에서 출발할 수 있게 출발지점을 무작위로 설정
    i = np.random.randint(0,env.reward.shape[0])
    j = np.random.randint(0,env.reward.shape[1])
    agent.set_pos([i,j])
    #에피소드의 수익을 초기화
    G = 0
    #감쇄율의 지수
    step = 0
    max_step = 100
    # 에피소드 생성
    for k in range(max_step):
        pos = agent.get_pos()
        action = np.random.randint(0,len(agent.action))
        observaetion, reward, done = env.move(agent, action)

        if first_visit:
            # 에피소드에 첫 방문한 상태인지 검사 :
            # visit[pos[0],pos[1]] == 0 : 첫 방문
            # visit[pos[0],pos[1]] == 1 : 중복 방문
            if visit[pos[0],pos[1]] == 0:
                # 에피소드가 끝날때까지 G를 계산
                G += gamma**(step) * reward
                # 방문 이력 표시
                visit[pos[0],pos[1]] = 1
                step += 1
                # 방문 이력 저장(상태, 행동, 보상)
                episode.append((pos,action, reward))
        else:
            G += gamma**(step) * reward
            step += 1
            episode.append((pos,action,reward))

        # 에피소드가 종료했다면 루프에서 탈출
        if done == True:
            break

    return i, j, G, episode

### 83~84

In [ ]:
# first-visit MC and every-visit MC prediction
np.random.seed(0)
# 환경, 에이전트를 초기화
env = Environment()
agent = Agent()

# 임의의 상태 가치 함수𝑉
v_table = np.zeros((env.reward.shape[0], env.reward.shape[1]))

# 상태별로 에피소드 출발횟수를 저장하는 테이블
v_start = np.zeros((env.reward.shape[0], env.reward.shape[1]))

# 상태별로 도착지점 도착횟수를 저장하는 테이블
v_success = np.zeros((env.reward.shape[0], env.reward.shape[1]))

# 𝑅𝑒𝑡𝑢𝑟𝑛(𝑠)←빈 리스트 (모든 s∈𝑆에 대해)
Return_s = [[[] for j in range(env.reward.shape[1])] for i in range(env.reward.shape[0])]

# 최대 에피소드 수를 지정
max_episode = 100000

# first visit 를 사용할지 every visit를 사용할 지 결정
# first_visit = True : first visit
# first_visit = False : every visit
first_visit = True
if first_visit:
    print("start first visit MC")
else :
    print("start every visit MC")
print()

for epi in tqdm(range(max_episode)):

    i,j,G,episode = generate_episode(env, agent, first_visit)

    # 수익 𝐺를 𝑅𝑒𝑡𝑢𝑟𝑛(𝑠)에 추가(append)
    Return_s[i][j].append(G)

    # 에피소드 발생 횟수 계산
    episode_count = len(Return_s[i][j])
    # 상태별 발생한 수익의 총합 계산
    total_G = np.sum(Return_s[i][j])
    # 상태별 발생한 수익의 평균 계산
    v_table[i,j] = total_G / episode_count

  # 도착지점에 도착(reward = 1)했는지 체크
    # episode[-1][2] : 에피소드 마지막 상태의 보상
    if episode[-1][2] == 1:
        v_success[i,j] += 1

# 에피소드 출발 횟수 저장
for i in range(env.reward.shape[0]):
    for j in range(env.reward.shape[1]):
        v_start[i,j] = len(Return_s[i][j])

print("V(s)")
show_v_table(np.round(v_table,2),env)
print("V_start_count(s)")
show_v_table(np.round(v_start,2),env)
print("V_success_pr(s)")
show_v_table(np.round(v_success/v_start,2),env)


start every visit MC



100%|████████████████████████████████████████████████████████████████████████| 100000/100000 [00:40<00:00, 2493.12it/s]


V(s)
+-----------------+-----------------+-----------------+
|                 |                 |                 |
|      -2.07      |      -1.62      |      -2.06      |
|                 |                 |                 |
+-----------------+-----------------+-----------------+
|                 |                 |                 |
|      -1.62      |      -1.12      |      -1.06      |
|                 |                 |                 |
+-----------------+-----------------+-----------------+
|                 |                 |                 |
|      -2.05      |      -1.05      |       1.00      |
|                 |                 |                 |
+-----------------+-----------------+-----------------+
V_start_count(s)
+-----------------+-----------------+-----------------+
|                 |                 |                 |
|   11302.00      |   11148.00      |   11021.00      |
|                 |                 |                 |
+-----------------+-------

### 88~90p

In [ ]:
#Incremental mean 을 이용하는 몬테카를로 Prediction 알고리즘
np.random.seed(0)
# 환경, 에이전트를 초기화
env = Environment()
agent = Agent()

# 임의의 상태 가치 함수𝑉
v_table = np.zeros((env.reward.shape[0], env.reward.shape[1]))

# 추가
# 상태를 방문한 횟수를 저장하는 테이블
v_visit = np.zeros((env.reward.shape[0], env.reward.shape[1]))

# 삭제
# # 𝑅𝑒𝑡𝑢𝑟𝑛(𝑠)←빈 리스트 (모든 s∈𝑆에 대해) :
# Return_s = [[[] for j in range(env.reward.shape[1])] for i in range(env.reward.shape[0])]

# 최대 에피소드 수와 에피소드 최대 길이지정
max_episode = 100000

# first visit을 사용할지 every visit을 사용할 지 결정
# first_visit = True : first visit
# first_visit = False : every visit
first_visit = True
if first_visit:
    print("start first visit MC")
else :
    print("start every visit MC")
print()

for epi in tqdm(range(max_episode)):

    i,j,G,episode = generate_episode(env, agent, first_visit)

    # 삭제
    # 수익 𝐺를 𝑅𝑒𝑡𝑢𝑟𝑛(𝑠)에 추가(append)
    #Return_s[i][j].append(G)

    # 에피소드 발생 횟수 계산
    #episode_count = len(Return_s[i][j])
    # 상태별 발생한 수익의 총합 계산
    #total_G = np.sum(Return_s[i][j])
    # 상태별 발생한 수익의 평균 계산
    #v_table[i,j] = total_G / episode_count

    #Return_length[i][j].append(len(episode))

    #if episode[-1][2] == 1:
    #    v_success[i,j] += 1

    # Incremental mean  평균을 계산
    v_visit[i,j] += 1
    v_table[i,j] += 1 / v_visit[i,j] * (G - v_table[i,j])


print("V(s)")
show_v_table(np.round(v_table,2),env)
print()

start first visit MC



100%|███████████████████████████████████████████████████████████████████████| 100000/100000 [00:03<00:00, 28141.46it/s]


V(s)
+-----------------+-----------------+-----------------+
|                 |                 |                 |
|      -2.07      |      -1.61      |      -2.06      |
|                 |                 |                 |
+-----------------+-----------------+-----------------+
|                 |                 |                 |
|      -1.62      |      -1.12      |      -1.06      |
|                 |                 |                 |
+-----------------+-----------------+-----------------+
|                 |                 |                 |
|      -2.05      |      -1.05      |       1.00      |
|                 |                 |                 |
+-----------------+-----------------+-----------------+



###  에피소드 분리를 이용하는 Firsit-visit 몬테카를로 방법과 Every-visit 몬테카를로 방법의 Prediction 알고리즘 92~93p

In [ ]:
def generate_episode(env, agent, first_visit):
    gamma = 0.09
    # 에피소드를 저장할 리스트
    episode = []
    # 이전에 방문여부 체크
    visit = np.zeros((env.reward.shape[0], env.reward.shape[1]))

    # 에이전트는 항상 (0,0)에서 출발
    i = 0
    j = 0
    agent.set_pos([i,j])
    #에피소드의 수익을 초기화
    G = 0
    #감쇄율의 지수
    step = 0
    max_step = 100
    # 에피소드 생성
    for k in range(max_step):
        pos = agent.get_pos()
        action = np.random.randint(0,len(agent.action))
        observaetion, reward, done = env.move(agent, action)

        if first_visit:
            # 에피소드에 첫 방문한 상태인지 검사 :
            # visit[pos[0],pos[1]] == 0 : 첫 방문
            # visit[pos[0],pos[1]] == 1 : 중복 방문
            if visit[pos[0],pos[1]] == 0:
                # 에피소드가 끝날때까지 G를 계산
                G += gamma**(step) * reward
                # 방문 이력 표시
                visit[pos[0],pos[1]] = 1
                step += 1
                # 방문 이력 저장(상태, 행동, 보상)
                episode.append((pos,action, reward))
        else:
            G += gamma**(step) * reward
            step += 1
            episode.append((pos,action, reward))

        # 에피소드가 종료했다면 루프에서 탈출
        if done == True:
            break

    return i, j, G, episode

#에피소드 분리를 이용하는 몬테카를로 Prediction 알고리즘
np.random.seed(0)
# 환경, 에이전트를 초기화
env = Environment()
agent = Agent()

# 임의의 상태 가치 함수𝑉
v_table = np.zeros((env.reward.shape[0], env.reward.shape[1]))

# 상태를 방문한 횟수를 저장하는 테이블
v_visit = np.zeros((env.reward.shape[0], env.reward.shape[1]))

# 최대 에피소드 수와 에피소드 최대 길이를 지정
max_episode = 99999

# first visit 를 사용할지 every visit를 사용할 지 결정
# first_visit = True : first visit
# first_visit = False : every visit
first_visit = True
if first_visit:
    print("start first visit MC")
else :
    print("start every visit MC")
print()

gamma = 0.09
for epi in tqdm(range(max_episode)):

    i,j,G,episode = generate_episode(env, agent, first_visit)
    for step_num in range(len(episode)):
        G = 0
        # episode[step_num][0][0] : step_num번째 방문한 상태의 x 좌표
        # episode[step_num][0][1] : step_num번째 방문한 상태의 y 좌표
        # episode[step_num][1] : step_num번째 상태에서 선택한 행동
        i = episode[step_num][0][0]
        j = episode[step_num][0][1]
        # 에피소드 시작점을 카운트
        v_visit[i,j] += 1

        # 서브 에피소드 (episode[step_num:])의 출발부터 끝까지 보수 G를 계산
        # k[2] : episode[step_num][2] 과 같으며 step_num 번째 받은 보상
        # step : 감쇄율
        for step, k in enumerate(episode[step_num:]):
            G += gamma**(step)*k[2]

        # Incremental mean  평균을 계산
        v_table[i,j] += 1 / v_visit[i,j] * (G - v_table[i,j])

print("V(s)")
show_v_table(np.round(v_table,2),env)
print("V_start_count(s)")
show_v_table(np.round(v_visit,2),env)


start first visit MC



100%|█████████████████████████████████████████████████████████████████████████| 99999/99999 [00:04<00:00, 21403.76it/s]


V(s)
+-----------------+-----------------+-----------------+
|                 |                 |                 |
|      -2.07      |      -1.58      |      -2.04      |
|                 |                 |                 |
+-----------------+-----------------+-----------------+
|                 |                 |                 |
|      -1.59      |      -1.10      |      -1.01      |
|                 |                 |                 |
+-----------------+-----------------+-----------------+
|                 |                 |                 |
|      -2.03      |      -1.06      |       1.00      |
|                 |                 |                 |
+-----------------+-----------------+-----------------+
V_start_count(s)
+-----------------+-----------------+-----------------+
|                 |                 |                 |
|   99999.00      |   29514.00      |   10121.00      |
|                 |                 |                 |
+-----------------+-------

### e-greedy 확률 변화 p.92~93

In [ ]:
policy = np.array((0.1,0.2,0.3,0.4))
up_list=[]
right_list=[]
down_list=[]
left_list=[]
print("episilon |   a1  |    a2   |    a3  |   a4  ")
for epsilon in reversed(range(0, 11)):
    epsilon /= 10
    up = epsilon / len(policy)
    right = epsilon / len(policy)
    down = epsilon / len(policy)
    left = 1 -  epsilon + epsilon / len(policy)
    print("{0:}      | {1:.3f} |  {2:.3f}  |  {3:.3f} |  {4:.3f}".format(np.round(epsilon,3),np.round(up,3),np.round(right,3),np.round(down,3),np.round(left,3)))
#     policy_list.append((up,right,down,left))
#     up_list.append(up)
#     right_list.append(right)
#     down_list.append(down)
#     left_list.append(left)

episilon |   a1  |    a2   |    a3  |   a4  
1.0      | 0.250 |  0.250  |  0.250 |  0.250
0.9      | 0.225 |  0.225  |  0.225 |  0.325
0.8      | 0.200 |  0.200  |  0.200 |  0.400
0.7      | 0.175 |  0.175  |  0.175 |  0.475
0.6      | 0.150 |  0.150  |  0.150 |  0.550
0.5      | 0.125 |  0.125  |  0.125 |  0.625
0.4      | 0.100 |  0.100  |  0.100 |  0.700
0.3      | 0.075 |  0.075  |  0.075 |  0.775
0.2      | 0.050 |  0.050  |  0.050 |  0.850
0.1      | 0.025 |  0.025  |  0.025 |  0.925
0.0      | 0.000 |  0.000  |  0.000 |  1.000


### 10주차 몬테카를로 control 알고리즘 99p~103p

In [ ]:
def generate_episode_with_policy(env, agent, first_visit, policy):
    gamma = 0.09
    # 에피소드를 저장할 리스트
    episode = []
    # 이전에 방문여부 체크
    visit = np.zeros((env.reward.shape[0], env.reward.shape[1],len(agent.action)))

    # 에이전트는 항상 (0,0)에서 출발
    i = 0
    j = 0
    agent.set_pos([i,j])
    #에피소드의 수익을 초기화
    G = 0
    #감쇄율의 지수
    step = 0
    max_step = 100
    # 에피소드 생성
    for k in range(max_step):
        pos = agent.get_pos()
        # 현재 상태의 정책을 이용해 행동을 선택한 후 이동
        action = np.random.choice(range(0,len(agent.action)), p=policy[pos[0],pos[1],:])
        observaetion, reward, done = env.move(agent, action)

        if first_visit:
            # 에피소드에 첫 방문한 상태인지 검사 :
            # visit[pos[0],pos[1]] == 0 : 첫 방문
            # visit[pos[0],pos[1]] == 1 : 중복 방문
            if visit[pos[0],pos[1],action] == 0:
                # 에피소드가 끝날때까지 G를 계산
                G += gamma**(step) * reward
                # 방문 이력 표시
                visit[pos[0],pos[1],action] = 1
                step += 1
                # 방문 이력 저장(상태, 행동, 보상)
                episode.append((pos,action, reward))
        else:
            G += gamma**(step) * reward
            step += 1
            episode.append((pos,action,reward))

        # 에피소드가 종료했다면 루프에서 탈출
        if done == True:
            break

    return i, j, G, episode

# ϵ-정책을 이용하는 몬테카를로 control 알고리즘
np.random.seed(0)
# 환경, 에이전트를 초기화
env = Environment()
agent = Agent()

# 모든 𝑠∈𝑆,𝑎∈𝐴(𝑆)에 대해 초기화:
# # 𝑄(𝑠,𝑎)←임의의 값 (행동 개수, 미로 세로, 미로 가로)
Q_table = np.random.rand(env.reward.shape[0], env.reward.shape[1],len(agent.action))
print("Initial Q(s,a)")
show_q_table(Q_table,env)

# 상태를 방문한 횟수를 저장하는 테이블
Q_visit = np.zeros((env.reward.shape[0], env.reward.shape[1],len(agent.action)))

# 미로 모든 상태에서 최적 행동을 저장하는 테이블
# 각 상태에서 Q 값이 가장 큰 행동을 선택 후 optimal_a 에 저장
optimal_a = np.zeros((env.reward.shape[0],env.reward.shape[1]))
for i in range(env.reward.shape[0]):
    for j in range(env.reward.shape[1]):
        optimal_a[i,j] = np.argmax(Q_table[i,j,:])
print("initial optimal_a")
show_policy(optimal_a,env)

# π(𝑠,𝑎)←임의의 𝜖−탐욕 정책
# 무작위로 행동을 선택하도록 지정
policy = np.zeros((env.reward.shape[0], env.reward.shape[1],len(agent.action)))

# 한 상태에서 가능한 확률의 합이 1이 되도록 계산
epsilon = 0.8
for i in range(env.reward.shape[0]):
    for j in range(env.reward.shape[1]):
        for k in range(len(agent.action)):
            if optimal_a[i,j] == k:
                policy[i,j,k] = 1 - epsilon + epsilon/len(agent.action)
            else:
                policy[i,j,k] = epsilon/len(agent.action)
print("Initial Policy")
show_q_table(policy,env)


# 최대 에피소드 수 길이를 지정
max_episode = 10000

# first visit 를 사용할지 every visit를 사용할 지 결정
# first_visit = True : first visit
# first_visit = False : every visit
first_visit = True
if first_visit:
    print("start first visit MC")
else :
    print("start every visit MC")
print()

gamma = 0.09
for epi in tqdm(range(max_episode)):
# for epi in range(max_episode):

    # π를 이용해서 에피소드 1개를 생성
    x,y,G,episode = generate_episode_with_policy(env, agent, first_visit, policy)

    for step_num in range(len(episode)):
        G = 0
        # episode[step_num][0][0] : step_num번째 방문한 상태의 x 좌표
        # episode[step_num][0][1] : step_num번째 방문한 상태의 y 좌표
        # episode[step_num][1] : step_num번째 상태에서 선택한 행동
        i = episode[step_num][0][0]
        j = episode[step_num][0][1]
        action = episode[step_num][1]

        # 에피소드 시작점을 카운트
        Q_visit[i,j,action] += 1

        # 서브 에피소드 (episode[step_num:])의 출발부터 끝까지 수익 G를 계산
        # k[2] : episode[step_num][2] 과 같으며 step_num 번째 받은 보상
        # step : 감쇄율
        for step, k in enumerate(episode[step_num:]):
            G += gamma**(step)*k[2]

        # Incremental mean : 𝑄(𝑠,𝑎)←𝑎𝑣𝑒𝑟𝑎𝑔𝑒(𝑅𝑒𝑡𝑢𝑟𝑛(𝑠,𝑎))
        Q_table[i,j,action] += 1 / Q_visit[i,j,action]*(G-Q_table[i,j,action])

    # (c) 에피소드 안의 각 s에 대해서 :
    # 미로 모든 상태에서 최적 행동을 저장할 공간 마련
    # 𝑎∗ ←argmax_a 𝑄(𝑠,𝑎)
    for i in range(env.reward.shape[0]):
        for j in range(env.reward.shape[1]):
            optimal_a[i,j] = np.argmax(Q_table[i,j,:])

    # 모든 𝑎∈𝐴(𝑆) 에 대해서 :
    # 새로 계산된 optimal_a 를 이용해서 행동 선택 확률 policy (π) 갱신
    epsilon = 1 - epi/max_episode

    for i in range(env.reward.shape[0]):
        for j in range(env.reward.shape[1]):
            for k in range(len(agent.action)):
                if optimal_a[i,j] == k:
                    policy[i,j,k] = 1 - epsilon + epsilon/len(agent.action)
                else:
                    policy[i,j,k] = epsilon/len(agent.action)

print("Final Q(s,a)")
show_q_table(Q_table,env)
print("Final policy")
show_q_table(policy,env)
print("Final optimal_a")
show_policy(optimal_a,env)

Initial Q(s,a)
+-----------------+-----------------+-----------------+
|      0.55       |      0.42       |      0.96       |
|  0.54      0.72 |  0.89      0.65 |  0.53      0.38 |
|      0.60       |      0.44       |      0.79       |
+-----------------+-----------------+-----------------+
|      0.57       |      0.02       |      0.98       |
|  0.09      0.93 |  0.87      0.83 |  0.78      0.80 |
|      0.07       |      0.78       |      0.46       |
+-----------------+-----------------+-----------------+
|      0.12       |      0.52       |      0.46       |
|  0.94      0.64 |  0.77      0.41 |  0.62      0.57 |
|      0.14       |      0.26       |      0.02       |
+-----------------+-----------------+-----------------+
initial optimal_a
+-----------------+-----------------+-----------------+
|                 |                 |                 |
|      →         |      ←         |      ↑         |
|                 |                 |                 |
+-----------------

100%|██████████████████████████████████████████████████████████████████████████| 10000/10000 [00:03<00:00, 3304.57it/s]


Final Q(s,a)
+-----------------+-----------------+-----------------+
|     -3.00       |     -3.00       |     -3.00       |
| -3.00     -1.13 | -1.17     -1.16 | -1.13     -3.00 |
|     -1.12       |     -1.09       |     -1.02       |
+-----------------+-----------------+-----------------+
|     -1.17       |     -1.13       |     -1.14       |
| -3.00     -1.09 | -1.12     -0.96 | -1.03     -3.00 |
|     -1.15       |     -1.02       |      1.09       |
+-----------------+-----------------+-----------------+
|     -1.14       |     -1.08       |      1.00       |
| -3.00     -1.01 | -1.14      1.09 |  1.00      1.00 |
|     -3.00       |     -3.00       |      1.00       |
+-----------------+-----------------+-----------------+
Final policy
+-----------------+-----------------+-----------------+
|      0.00       |      0.00       |      0.00       |
|  0.00      0.00 |  0.00      0.00 |  0.00      0.00 |
|      1.00       |      1.00       |      1.00       |
+-----------------+---